# Predicción Multi-Clase de Cirrosis - Código de Ejecución

In [1]:
# Importaciones
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from janitor import clean_names
import cloudpickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler

def ejecucion(df):
    df = clean_names(df[['N_Days', 'Drug', 'Age', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders',
           'Edema', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper', 'Alk_Phos',
           'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin', 'Stage']])
    
    categoricas = ['drug', 'sex', 'ascites', 'hepatomegaly', 'spiders', 'edema', 'stage']
    numericas = ['n_days', 'age', 'bilirubin', 'cholesterol', 'albumin', 'copper', 'alk_phos', 'sgot', 'tryglicerides', 'platelets', 'prothrombin']
    
    var_ohe = categoricas[:-1]
    
    # Aplicar OneHotEncoder a las variables nominales
    ohe = OneHotEncoder(sparse = False, handle_unknown='ignore', drop='first')
    cat_ohe = ohe.fit_transform(df[var_ohe])
    cat_ohe = pd.DataFrame(cat_ohe, columns = ohe.get_feature_names_out())
    
    # Aplicar OrdinalEncoder a la variable ordinal
    encoder = OrdinalEncoder()
    df['stage_enc'] = encoder.fit_transform(df[['stage']])
    
    df = df.drop(columns = categoricas)
    df = pd.concat([df, cat_ohe], axis = 1)
    
    # La única variable que he creado
    df['sum_cat'] = df[['ascites_Y', 'hepatomegaly_Y', 'spiders_Y']].sum(axis=1)
    
    # Reescalado
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df)
    df = pd.DataFrame(df_scaled, columns=df.columns.to_list())
    
    # Cargar el modelo
    with open('modelo_final.pkl', 'rb') as archivo:
        modelo = cloudpickle.load(archivo)
    
    submission = pd.DataFrame(modelo.predict_proba(df))
    submission.columns = ['Status_C', 'Status_Cl', 'Status_D']
    return submission

In [2]:
ejecucion(pd.read_csv('test.csv'))

,Status_C,Status_Cl,Status_D
0,0.390483,0.024700,0.584817
1,0.602562,0.105816,0.291622
2,0.053652,0.019484,0.926864
3,0.970881,0.003118,0.026000
4,0.862957,0.049883,0.087160
...,...,...,...
5266,0.898979,0.055231,0.045790
5267,0.968070,0.004663,0.027267
5268,0.903073,0.010414,0.086513
5269,0.984692,0.005725,0.009583
